In [16]:
# Simulador de Filtros
# Autores: Ramiro Belsito y Ignacio Sammartino
import re
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import math

def ENG( valor: float, unidad: str) -> str:
    """
    Convierte un valor de resistencia o capacitancia a notación de ingeniería.
    
    Parámetros:
    - valor: El valor numérico (float)
    - unidad: 'Ohm' para resistencias o 'F' para capacitores

    Retorna:
    - str con el valor en notación de ingeniería (ej. 4.7 kOhm, 10 nF)
    """
    prefijos = {
        -12: 'p',
        -9: 'n',
        -6: 'µ',
        -3: 'm',
         0: '',
         3: 'k',
         6: 'M',
         9: 'G',
    }

    if valor == 0:
        return f"0 {unidad}"
    
    exponente = int(math.floor(math.log10(abs(valor)) // 3 * 3))
    exponente = max(min(exponente, 9), -12)  # limitar a prefijos comunes
    valor_normalizado = valor / (10 ** exponente)
    
    prefijo = prefijos.get(exponente, f"e{exponente}")
    
    return f"{valor_normalizado:.3g} {prefijo}{unidad}"

In [ ]:
TipoDeFiltro = input("Ingrese el tipo de filtro (PasaBajo, PasaAlto, PasaBanda, RechazaBanda): ")

if re.search("alto", TipoDeFiltro, re.IGNORECASE) or re.search("hp", TipoDeFiltro, re.IGNORECASE):
    TipoDeFiltro = "HP"
elif re.search("bajo", TipoDeFiltro, re.IGNORECASE) or re.search("lp", TipoDeFiltro, re.IGNORECASE):
    TipoDeFiltro = "LP"
else:
    if re.search("rechaza", TipoDeFiltro, re.IGNORECASE) or re.search("br", TipoDeFiltro, re.IGNORECASE):
        TipoDeFiltro = "BR"
    elif re.search("banda", TipoDeFiltro, re.IGNORECASE) or re.search("bp", TipoDeFiltro, re.IGNORECASE):
        TipoDeFiltro = "BP"
    else:
        print("Error: Tipo de filtro no reconocido.")
        exit(1)

check = True

if TipoDeFiltro == "LP":
    fp = float(input("Ingrese la frecuencia de paso (Hz): "))
    fa = float(input("Ingrese la frecuencia de rechazo (Hz): "))
    Ap = float(input("Ingrese la atenuación en banda de paso (dB): "))
    Aa = float(input("Ingrese la atenuación en banda de rechazo (dB): "))
    faN = fa/fp
    fpN = 1
    if not ((fp < fa) and (Ap < Aa)):
        print("Error: Los parametros no cumplen con la condición de diseño.")
        check = False
    else:
        print("fp = ", ENG(fp, "Hz"))
        print("fa = ", ENG(fa, "Hz"))
        print("Ap = ", Ap, "dB")
        print("Aa = ", Aa, "dB")
        print("Las frecuencias cumplen con la condición de diseño.")
        
elif TipoDeFiltro == "HP":
    fp = float(input("Ingrese la frecuencia de paso (Hz): "))
    fa = float(input("Ingrese la frecuencia de rechazo (Hz): "))
    Ap = float(input("Ingrese la atenuación en banda de paso (dB): "))
    Aa = float(input("Ingrese la atenuación en banda de rechazo (dB): "))
    faN = fp/fa
    fpN = 1
    if not ((fa < fp) and (Ap < Aa)):
        print("Error: Los parametros no cumplen con la condición de diseño.")
        check = False
    else: 
        print("fp = ", ENG(fp, "Hz"))
        print("fa = ", ENG(fa, "Hz"))
        print("Ap = ", Ap, "dB")
        print("Aa = ", Aa, "dB")
        print("Las frecuencias cumplen con la condición de diseño.")
elif TipoDeFiltro == "BP":
    famin = float(input("Ingrese la frecuencia de rechazo inferior (Hz): "))
    fpmin = float(input("Ingrese la frecuencia de paso inferior (Hz): "))
    f0 = float(input("Ingrese la frecuencia central (Hz): "))
    fpmax = f0**2 / fpmin
    famax = f0**2 / famin
    Ap = float(input("Ingrese la atenuación en banda de paso (dB): "))
    Aa = float(input("Ingrese la atenuación en banda de rechazo (dB): "))
    B = fpmax - fpmin
    BA = famax - famin
    faN = BA/B
    fpN = 1
    if not ((famin < fpmin < f0 < fpmax < famax) and (Ap < Aa)):
        print("Error: Los parametros no cumplen con la condición de diseño.")
        check = False
    else:
        print("famin = ", ENG(famin, "Hz"))
        print("fpmin = ", ENG(fpmin, "Hz"))
        print("f0 = ", ENG(f0, "Hz"))
        print("fpmax = ", ENG(fpmax, "Hz"))
        print("famax = ", ENG(famax, "Hz"))
        print("Ap = ", Ap, "dB")
        print("Aa = ", Aa, "dB")
        print("Las frecuencias cumplen con la condición de diseño.")
elif TipoDeFiltro == "BR":
    fpmin = float(input("Ingrese la frecuencia de paso inferior (Hz): "))
    famin = float(input("Ingrese la frecuencia de atenuacion inferior (Hz): "))
    f0 = float(input("Ingrese la frecuencia central (Hz): "))
    fpmax = f0**2 / fpmin
    famax = f0**2 / famin
    Ap = float(input("Ingrese la atenuación en banda de paso (dB): "))
    Aa = float(input("Ingrese la atenuación en banda de rechazo (dB): "))
    B = fpmax - fpmin
    BA = famax - famin
    faN = B/BA
    fpN = 1
    if not ((fpmin < famin < f0 < famax < fpmax) and (Ap < Aa)):
        print("Error: Las frecuencias no cumplen con la condición de diseño.")
        check = False
    else:
        print("fpmin = ", ENG(fpmin, "Hz"))
        print("famin = ", ENG(famin, "Hz"))
        print("f0 = ", ENG(f0, "Hz"))
        print("famax = ", ENG(fpmax, "Hz"))
        print("fpmax = ", ENG(famax, "Hz"))
        print("Ap = ", Ap, "dB")
        print("Aa = ", Aa, "dB")
        print("Las frecuencias cumplen con la condición de diseño.")

if not check:
    exit(1)



    



fpmin =  727 Hz
famin =  2.16 kHz
f0 =  2.4 kHz
famax =  7.92 kHz
fpmax =  2.67 kHz
Ap =  0.5 dB
Aa =  40.0 dB
Las frecuencias cumplen con la condición de diseño.
